# Pipelining with Titanic Data

### Data loading ...

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#### Some preprocessing of the data

It is likely that `PassengerId` or `Ticket` number don't affect on passenger survillance. So, lets drop these columns at the beggining.

In [4]:
combined = pd.concat([train, test]).reset_index(drop=True)

In [5]:
combined.shape


(1309, 12)

## Utility functions and classes

In [6]:
# self explanatory function, returns first name, title and the last name of a passenger
def parse_name(s): 
    a, b = s.split(',')
    family_name = a.strip()
    title = b.split('.')[0].strip()
    first_name = b.split('.')[1].split()[0].strip()
    return (first_name, title, family_name)


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from IPython.core.display import display, HTML

class AbstractPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise Exception("Input data should be a DataFrame instance")
        return self


class DropColumns(AbstractPreprocessor):
    '''
    Drops specified columns from a DataFrame.
    
    Input data assumed to be a DataFrame.
    
    Usage
    -----
        dropper = DropColumns(names=['PassengerId', 'Ticket'])
        dropped_df = dropper.fit_transform(source_dataframe)
    '''
    
    def __init__(self, names=[]):
        self.names = names
    
    def transform(self, X, y=None):
        filtered = [col for col in self.names if col in X.columns]
        return X.drop(filtered, axis=1)

class AddFirstNameColumn(AbstractPreprocessor):

    def transform(self, X, y=None):
        result = X.copy()
        result['first_name'] = X.loc[:, 'Name'].apply(lambda x: parse_name(x)[0].replace('(', '').replace(')', ''))
        return result

class AddFamilyNameColumn(AbstractPreprocessor):
    def transform(self, X, y=None):
        result = X.copy()
        result['family_name'] = X.loc[:, 'Name'].apply(lambda x: parse_name(x)[-1])
        return result
                
class AddTitleColumn(AbstractPreprocessor):

    def transform(self, X, y=None):
        result = X.copy()
        result['title'] = X.loc[:, 'Name'].apply(lambda x: parse_name(x)[1])
        return result   

class AddFamilySize(AbstractPreprocessor):
    def transform(self, X, y=None):
        _X = X.copy()
        _X['family_size'] = X['SibSp'] + X['Parch'] + 1
        return _X

class AddIsAlone(AbstractPreprocessor):
    def transform(self, X, y=None):
        _X = X.copy()
        _X['is_alone'] = 1
        _X.loc[_X.family_size > 1,'is_alone'] = 0
        return _X

class OneHotEncodeAndDrop(AbstractPreprocessor):
    
    def __init__(self, name=None):
        self.name = name
    
    def transform(self, X, y=None):
        aux = pd.get_dummies(X.loc[:, self.name], prefix=self.name)
        return pd.concat([X.drop(self.name, axis=1), aux], axis=1)

class LabelEncodeAndDrop(AbstractPreprocessor):
    
    def __init__(self, name=None):
        self.name = name
        self.enc = LabelEncoder()
    
    def fit(self, X, y=None):
        self.enc.fit(X.loc[:, self.name].values)
        return self
        
    def transform(self, X, y=None):
        _X = X.copy()
        new_values = self.enc.transform(_X.loc[:, self.name].values)
        _X.loc[:, self.name] = new_values
        return _X

    
class CombineCategoricalValues(AbstractPreprocessor):
    def __init__(self, name=None, rule={'what': [], 'to': None}):
        self.name = name
        self.rule = rule
        
    def transform(self, X, y=None):
        mask = X.loc[:, self.name].isin(self.rule['what'])
        _X = X.copy()
        _X.loc[:, self.name][mask] = self.rule['to']
        return _X
        
class DropByValue(AbstractPreprocessor):
    def __init__(self, name=None, value=None):
        self.name = name
        self.value = value
    
    def transform(self, X, y=None):
        return X[X.loc[:, self.name] != self.value]
    
class GetCategoriesAndDrop(AbstractPreprocessor):
    def __init__(self, name=None, bins=None):
        self.name = name
        self.bins = bins
    
    def transform(self, X, y=None):
        aux = pd.cut(X.loc[:, self.name], bins=self.bins, labels=False)
        return X.drop(self.name, axis=1).join(pd.get_dummies(aux, prefix=self.name))
   
    
from sklearn.ensemble import RandomForestRegressor
class FillNaValues(AbstractPreprocessor):

    def __init__(self, name=None, train=None, n_features=None,
                 clf=RandomForestRegressor()):
        self.train = train
        self.name = name
        self.clf = clf
        self.n_features = n_features
    
    def transform(self, X, y=None):

        if self.name is None: 
            return X

        if X.loc[:, self.name].isnull().sum() == 0:
            return X
        
        _train = self.train.copy() if self.train is not None else X.copy()
        null_mask = _train[self.name].isnull()
        y = _train[self.name][~null_mask]
        _train = _train.drop(self.name, axis=1)
        
        n_features = int(pd.np.ceil(X.shape[1] * 0.3) or self.n_features)
        
        encoders = dict()
        for key in _train.columns.tolist():
            if not pd.np.issubdtype(_train[key].dtype, pd.np.number):
                _train.loc[_train[key].isnull(), key]  = 'N-a-N'
                le = LabelEncoder()
                _train[key] = le.fit_transform(_train[key])
                encoders[key] = le
            else:
                if any(_train[key].isnull()):
                    _train['%s_nan' % key] = 0.0
                    _train.loc[_train[key].isnull(), '%s_nan' % key] = 1.0
                    _train.loc[_train[key].isnull(), key] = _train.loc[~_train[key].isnull(), key].median()

        self.clf.fit(_train[~null_mask], y)
        
        # dropping features
        if hasattr(self.clf, 'feature_importances_'):
            # drop columns and retrain classifier
            indices = pd.np.argsort(self.clf.feature_importances_)[::-1]
            features_to_drop = _train.columns[indices].values.tolist()[n_features:]
            self.clf.fit(_train.drop(features_to_drop, axis=1)[~null_mask], y)
        else:
            features_to_drop = []
            
        _X = X.copy()
        for key in _train.columns:
            if key not in _X.columns:
                _X.loc[:, key] = 0.0
        _X = _X[_train.columns]
        for key in encoders.keys():
            if not pd.np.issubdtype(_X[key].dtype, pd.np.number):
                _X.loc[_X[key].isnull(), key]  = 'N-a-N'
                _X[key] = encoders[key].transform(_X[key])
            else:
                if any(_X[key].isnull()):
                    _X['%s_nan' % key] = 0.0
                    _X.loc[_X[key].isnull(), '%s_nan' % key] = 1.0
                    _X.loc[_X[key].isnull(), key] = X.loc[~_X[key].isnull(), key].median()
        
        na_replacements = self.clf.predict(_X.drop(features_to_drop, axis=1)[null_mask])
        result = X.copy()
        result.loc[null_mask, self.name] = na_replacements
        return result

class FillNaEmbarkedSimple(AbstractPreprocessor):
    
    def transform(self, X, y=None):
        _X = X.copy()
        _X['Embarked'] = _X['Embarked'].fillna(_X.Embarked.dropna().mode()[0])
        return _X
    
class DropRows(AbstractPreprocessor):

    def __init__(self, condition=None):
        self.condition = condition #condition depends on (X, y) and 
        # returns boolean array of the same length as X

    def fit(self, X, y=None):
        return self
    
    def trasform(self, X, y=None):
        if self.condition is not None:
            return X[self.condition(X, y)]
        else:
            return X


class ShowDataHead(AbstractPreprocessor):
    def fit(self, X, y=None):
        display(HTML(X.head().to_html()))
        return self
        
    def transform(self, X, y=None):
        return X

# Preprocessing steps (feature engeneering)

In [7]:
from sklearn.pipeline import Pipeline
preprocessing_steps = [('drop_columns', DropColumns(names=['Ticket', 'PassengerId', 'Cabin', 'Survived'])),
                       ('add_title', AddTitleColumn()),
                       ('add_first_name', AddFirstNameColumn()),
                       ('add_family_name', AddFamilyNameColumn()),
                       ('add_family_size', AddFamilySize()),
                       ('drop_name', DropColumns(names=['Name'])),
                       ('encode_sex', LabelEncodeAndDrop(name='Sex')),
                       ('fillna_embarked', FillNaEmbarkedSimple()),
                       ('encode_embarked', LabelEncodeAndDrop(name='Embarked')),
                       
                       # predicting nan-values
                       ('predict_fare', FillNaValues(name='Fare')),
                       ('predict_age', FillNaValues(name='Age')),
                     
                       # combine & encode titles
                       ('combine_title_Mrs', CombineCategoricalValues(name='title', rule={'what': ['Dona', 'Lady', 'Mme', 'the Countess'], 'to': 'Mrs'})),
                       ('combine_title_Mr', CombineCategoricalValues(name='title', rule={'what': ['Master', 'Jonkheer', 'Major', 'Col', 'Sir', 'Rev', 'Don', 'Capt', 'Dr'], 'to': 'Mr'})),
                       ('combine_title_Miss', CombineCategoricalValues(name='title', rule={'what': ['Mlle', 'Ms'], 'to': 'Miss'})),
                       ('encode_title', LabelEncodeAndDrop(name='title')),
                       
                       #
                       ('add_is_alone', AddIsAlone()),
                       
                       #drop first name, family name
                       ('drop_last', DropColumns(names=['first_name','family_name', 'family_size'])),
                      ]

preprocessing_pipeline = Pipeline(steps=preprocessing_steps)
combined_processed = preprocessing_pipeline.fit_transform(combined)

/home/dmitry/.pyenv/versions/3.5.4/envs/sci/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/dmitry/.pyenv/versions/3.5.4/envs/sci/lib/python3.5/site-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# It seems that all is ready. Lets make a classifier!

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis


In [9]:
train_processed = combined_processed.iloc[:train.shape[0]]
test_processed = combined_processed.iloc[train.shape[0]:]
y = train.Survived.values.astype(int)
X = train_processed.values

In [10]:
classifiers = [{'clf': LogisticRegression(), 'params': {'penalty':('l1', 'l2'), 'C': pd.np.linspace(1.e-6, 3, 20)}}, 
               {'clf': RandomForestClassifier(), 'params':{'n_estimators':[10, 50, 100, 500], 'max_depth': [2, 3, 5, 7, 11], 
                                                          'max_features': ['auto', 'log2', 'sqrt']}},
               {'clf': KNeighborsClassifier(), 'params': {'n_neighbors': [1, 3, 5, 7], 'p': [1, 2]}},
               {'clf': GaussianNB(), 'params': {'priors': [[0.7, 0.3], [0.8, 0.2]]}},
               {'clf': AdaBoostClassifier(), 'params': {'base_estimator': (DecisionTreeClassifier(max_depth=1),
                                                                           DecisionTreeClassifier(max_depth=2),
                                                                           DecisionTreeClassifier(max_depth=3),
                                                                           DecisionTreeClassifier(max_depth=5),
                                                                           DecisionTreeClassifier(max_depth=7)),
                                                       'n_estimators': [20, 50, 100, 200],
                                                       'algorithm': ['SAMME', 'SAMME.R'],
                                                       'learning_rate': (2.0, 1.0, 0.5, 0.05)}},
               {'clf': GradientBoostingClassifier(), 'params': {'loss': ('deviance', 'exponential'),
                                                                'learning_rate': (0.5, 0.1, 0.05, 0.01),
                                                                'n_estimators': [20, 50, 100, 200],
                                                                'subsample': [1.0, 0.9],
                                                                'max_depth': [1,2,3]}},
               {'clf': LinearDiscriminantAnalysis(), 'params': {'priors': [[0.68, 0.32]], 'n_components': [2, 4, 6]}},
               {'clf': QuadraticDiscriminantAnalysis(), 'params': {'priors': [[0.68, 0.32]]}},
               #{'clf': SVC(gamma='scale'), 'params': {'kernel':('rbf', 'linear'), 'C': [1.0, 10.0, 100.0, 1000.0]}},
              ]
results = dict()
for c in classifiers:
    print("Performing grid searching for ", c['clf'])
    clf = GridSearchCV(c['clf'], c['params'], cv=5, scoring='balanced_accuracy', verbose=True, n_jobs=-1)
    clf.fit(X, y)
    results[c['clf']] = clf
    print("Best score is ", clf.best_score_)

Performing grid searching for  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.9s finished
/home/dmitry/.pyenv/versions/3.5.4/envs/sci/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best score is  0.7810757177038836
Performing grid searching for  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   28.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best score is  0.8055130183681055
Performing grid searching for  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
Fitting 5 folds for each of 8 candidates, totalling 40 fits


...

[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   51.2s finished
/home/dmitry/.pyenv/versions/3.5.4/envs/sci/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best score is  0.8092941565353926
Performing grid searching for  GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done 443 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:   21.9s finished


Best score is  0.813375361346313
Performing grid searching for  LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best score is  0.7737764666994019
Performing grid searching for  QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best score is  0.7984102358234704


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [11]:
best_classifiers = list(map(lambda x: (x[0], x[1]), sorted([(k.__class__.__name__, v.best_estimator_, v) for k, v in results.items()], key=lambda x: x[-1].best_score_, reverse=True)[:3]))
best = VotingClassifier(best_classifiers, n_jobs=-1)
best.fit(X, y)
print("Estimated score is ", cross_val_score(best, X, y, cv=10).mean())

Estimated score is  0.8317319260015891


In [12]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": best.predict(test_processed.values).astype(int)
    })
submission.to_csv('./output/submission.csv', index=False)

In [13]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0
